In [1]:
from keras.applications import ResNet50V2
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from glob import glob
import tensorflow as tf

train = pd.read_csv('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\train_new4.csv')
print(train.head())

images = glob("D:/Python/Pycharm/PycharmProjects/IPFinal/train_6/*.jpg")
x1 = np.array([np.array(image.load_img(fname, target_size=(224, 224, 3))) for fname in tqdm(images)])

print(x1.shape)

# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(x1, y, random_state=42, test_size=0.2, stratify=y)
print(X_train.shape)
print(X_test.shape)
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

# creating the base model of pre-trained model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

print(X_train.shape)
print(X_test.shape)

# normalizing the pixel values
max1 = X_train.max()
X_train = X_train/max1
X_test = X_test/max1

print(X_train.shape)
print(X_test.shape)

# shape of images
print(X_train.shape)

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(6, activation='softmax')
])

model.summary()
print("model Constructed")
# defining a function to save the weights of best model
mcp_save = ModelCheckpoint('D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\FTry1.hdf5', save_best_only=True, monitor='val_loss', verbose=2, mode='max')

# compiling the model
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
print("model compiled")
# training the model
model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=32)
print("model fitted")
# Saving the model for Future Inferences

model_json = model.to_json()
with open("D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\modelsum2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("D:\\Python\\Pycharm\\PycharmProjects\\IPFinal\\FTry2.h5")

print("mode and weights stored.")



  0%|▍                                                                              | 21/4288 [00:00<00:23, 185.31it/s]

                                    image         class
0   v_BabyCrawling_g08_c01.avi_frame1.jpg  BabyCrawling
1  v_BabyCrawling_g08_c01.avi_frame11.jpg  BabyCrawling
2  v_BabyCrawling_g08_c01.avi_frame13.jpg  BabyCrawling
3  v_BabyCrawling_g08_c01.avi_frame15.jpg  BabyCrawling
4  v_BabyCrawling_g08_c01.avi_frame17.jpg  BabyCrawling


100%|█████████████████████████████████████████████████████████████████████████████| 4288/4288 [00:19<00:00, 217.82it/s]


(4288, 224, 224, 3)
(3430, 224, 224, 3)
(858, 224, 224, 3)
(3430, 224, 224, 3)
(858, 224, 224, 3)
(3430, 224, 224, 3)
(858, 224, 224, 3)
(3430, 224, 224, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               25690368  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (N